In [ ]:
%pip install tensorflow
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install scikit-learn
%pip install nltk

In [ ]:
pip install tensorflow

In [107]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [108]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [109]:
# I need to split train data into labels and features
train_labels = train_data['label'].to_numpy()
train_features = train_data['text'].to_numpy()

In [110]:
test_labels = test_data['label'].to_numpy()
test_features = test_data['text'].to_numpy()

In [ ]:
#remove stopwords from train_features
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

for i in range(len(train_features)):
    word_tokens = train_features[i].split()
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    train_features[i] = ' '.join(filtered_sentence)

for i in range(len(test_features)):
    word_tokens = test_features[i].split()
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    test_features[i] = ' '.join(filtered_sentence)

print(train_features)
print(test_features)

In [ ]:
# Tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
for i in range(len(train_features)):
    train_features[i] = train_features[i].lower()[:100]
for i in range(len(test_features)):
    test_features[i] = test_features[i].lower()[:100]
tokenizer.fit_on_texts(train_features)
word_index = tokenizer.word_index
print(word_index)

train_sequences = tokenizer.texts_to_sequences(train_features)
train_padded = pad_sequences(train_sequences, padding='post').astype(np.float32)

test_sequences = tokenizer.texts_to_sequences(test_features)
test_padded = pad_sequences(test_sequences, padding='post').astype(np.float32)

In [ ]:
#print distinct labels
print(np.unique(train_labels))
print(np.unique(test_labels))

# convert labels to distinct integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)
train_labels = label_encoder.transform(train_labels)
test_labels = label_encoder.transform(test_labels)

In [ ]:
print(train_padded.shape)
#pad train_padded to have same shape as test_padded
test_padded = np.pad(test_padded, ((0,0), (0, 25 - test_padded.shape[1])), 'constant')
print(test_padded.shape)
print(train_padded.shape)

In [58]:
#normalize the data
train_padded = train_padded / 10000
test_padded = test_padded / 10000

In [ ]:
print(train_padded)
print(train_labels)
print(test_labels)

In [75]:
# From train data select only those entries that have label 1 or 5
train_padded = train_padded[np.where((train_labels == 1) | (train_labels == 5))]
train_labels = train_labels[np.where((train_labels == 1) | (train_labels == 5))]

In [ ]:
#Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=25),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')  # Or adjust for multiclass
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 25))
model.summary()


In [ ]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=30, validation_data=(test_padded, test_labels), verbose=2)
